In [ ]:
import os
import subprocess
from pathlib import Path
from collections import defaultdict
import pandas as pd

In [ ]:
hyp_path = Path("path/to/your/data/analysis/hyp") ###
ref_path = Path("path/to/your/data/analysis/ref") ###
out_path = Path("path/to/your/data/analysis/results/1. bias") ###

In [ ]:
def extract_speaker_metadata(models_root: Path, stm_path: Path) -> dict:
    """
    Extracts speaker metadata from .sys and .stm files and returns a pandas DataFrame.

    Args:
        models_root (Path): Path to folder containing model subfolders with .sys files.
        stm_path (Path): Path to folder containing reference .stm files.

    Returns:
        pd.DataFrame: Rows with columns ['sys_filename', 'speaker_id', 'gender', 'age', 
                       'first_language', 'proficiency', 'accent_region', 'native', 'region']
    """
    speaker_metadata = defaultdict(dict)

    for model_dir in models_root.iterdir():
        if not model_dir.is_dir():
            continue

        for subset_dir in model_dir.iterdir():
            if not subset_dir.is_dir():
                continue
            #sets 1-4 for jasmin
            if not any(digit in subset_dir.stem.split('_')[1] for digit in ['1', '2', '3', '4']):
                continue
            print(f"Processing subset directory: {subset_dir.name}")

            for sys_file in subset_dir.glob("*.ctm.sys"):
                speaker_ids = []

                #extract speaker ids from .sys file
                with sys_file.open("r", encoding="utf-8") as f:
                    for line in f:
                        if line.startswith("|") and "|" in line[1:]:
                            columns = line.strip().split("|")
                            if len(columns) > 1:
                                candidate = columns[1].strip()
                                if candidate and candidate.startswith(("spk", "n00", "n10", "v00")):
                                    speaker_ids.append(candidate)
                #print(f"Found {len(speaker_ids)} speakers in {sys_file.name}")
                #print(f"Speaker IDs: {speaker_ids}")

                #find corresponding .stm file
                sys_stem = sys_file.name.replace(".ctm.sys", "")
                sys_stem_parts = sys_stem.split("_")
                if len(sys_stem_parts) < 2:
                    continue
                stm_id = sys_stem_parts[1]
                stm_file = stm_path / f"{stm_id}_reference.stm"

                if not stm_file.exists():
                    print(f"Warning: Corresponding .stm file not found for {sys_file.name}")
                    continue

                #extract metadata strings per speaker from .stm file
                with stm_file.open("r", encoding="utf-8") as f:
                    for line in f:
                        parts = line.strip().split()
                        if len(parts) < 6:
                            continue
                        speaker = parts[2]
                        meta_start = line.find('<')
                        meta_end = line.find('>', meta_start)
                        if meta_start != -1 and meta_end != -1 and speaker.lower() in speaker_ids:
                            metadata = line[meta_start + 1:meta_end]
                            speaker_metadata[sys_file.name][speaker] = metadata

    #parse metadata strings into structured data
    rows = []
    for sys_filename, speakers_meta in speaker_metadata.items():
        for spk, meta_str in speakers_meta.items():
            try:
                gender, age, first_lang, proficiency, accent_region = meta_str.split(",")
                age = float(age)
            except Exception as e:
                print(f"Error parsing metadata '{meta_str}' for speaker '{spk}' in '{sys_filename}': {e}")
                continue

            #'dut' for dutch, 'dia' for dialect, 'tus' for tussentaal. all are native dutch speakers
            if first_lang not in ["dut", "dia", "tus"]:
                native = "no"
            elif first_lang in ["dut", "dia", "tus"] and proficiency in ["A1", "A2", "B1", "B2"]:
                native = "no"
            elif first_lang in ["dut", "dia", "tus"] and proficiency in ["C1", "C2", "unknown"]:
                native = "yes"

            if accent_region.startswith("N"):
                region = "Dutch"
            elif accent_region.startswith("FL"):
                region = "Flemish"
            else:
                region = "Other"

            #since some speakers appear in both the read and spontaneous set, we add 
            #the style to the speaker id to prevent accidental overlap
            if any(digit in sys_filename.split('_')[1] for digit in ['1', '2']):
                spk = spk + 'r'
            elif any(digit in sys_filename.split('_')[1] for digit in ['3', '4']):
                spk = spk + 's'

            rows.append({
                "sys_filename": sys_filename,
                "speaker_id": spk,
                "gender": gender,
                "age": age,
                "first_language": first_lang,
                "proficiency": proficiency,
                "accent_region": accent_region,
                "native": native,
                "region": region
            })

    return pd.DataFrame(rows)


In [ ]:
def extract_speaker_performance(models_root: Path) -> dict:
    #extracts number of words and WER per speaker from each .sys file in model folders
    speaker_performance = defaultdict(dict)

    for model_dir in models_root.iterdir():
        if not model_dir.is_dir():
            continue

        for subset_dir in model_dir.iterdir():
            if not subset_dir.is_dir():
                continue
            if not any(digit in subset_dir.stem.split('_')[1] for digit in ['1', '2', '3', '4']):
                continue

            for sys_file in subset_dir.glob("*.ctm.sys"):
                current_sys = sys_file.name

                if any(digit in current_sys.split('_')[1] for digit in ['1', '2']):
                    suffix = 'r'
                elif any(digit in current_sys.split('_')[1] for digit in ['3', '4']):
                    suffix = 's'

                with sys_file.open("r", encoding="utf-8") as f:
                    for line in f:
                        #skip header, footer, or empty lines
                        if not line.startswith("|") or "---" in line or line.count("|") < 3:
                            continue

                        parts = [p.strip() for p in line.strip().strip("|").split("|")]
                        if len(parts) < 3:
                            continue

                        speaker_id = parts[0] + suffix
                        if not (speaker_id.lower().startswith("n") or speaker_id.lower().startswith("v")):
                            continue

                        try:
                            word_stats = parts[1].split()
                            err_stats = parts[2].split()

                            num_words = int(word_stats[1])  # #wrd is second in word_stats
                            wer = float(err_stats[4])       # err is fifth in err_stats

                            speaker_performance[current_sys][speaker_id] = {
                                "num_words": num_words,
                                "wer": wer
                            }
                        except (ValueError, IndexError):
                            continue 

    return speaker_performance

In [ ]:
metadata = extract_speaker_metadata(out_path, ref_path)
speaker_performance = extract_speaker_performance(out_path)

#print(speaker_performance)

In [ ]:
def highlight_extremes(dataframe):
    styles = pd.DataFrame('background-color: white; color: black', index=dataframe.index, columns=dataframe.columns)

    avg_row_mask = dataframe.index.get_level_values("Age Bin") == "Avg"
    avg_cols = [col for col in dataframe.columns if col.endswith("_Avg")]

    #gray out avg rows and columns
    styles.loc[avg_row_mask] = 'background-color: #dddddd; color: black; font-weight: bold'
    for col in avg_cols:
        styles[col] = 'background-color: #dddddd; color: black; font-weight: bold'

    #column-wise min/max highlighting (excluding Avg rows)
    for col in dataframe.columns:
        if col in avg_cols:
            continue

        col_values = dataframe.loc[~avg_row_mask, col]
        min_val = col_values.min(skipna=True)
        max_val = col_values.max(skipna=True)

        for idx in dataframe.index:
            if pd.isna(dataframe.at[idx, col]) or avg_row_mask[dataframe.index.get_loc(idx)]:
                continue
            if dataframe.at[idx, col] == min_val:
                styles.at[idx, col] = 'background-color: #cce5cc; color: black'
            elif dataframe.at[idx, col] == max_val:
                styles.at[idx, col] = 'background-color: #f5cccc; color: black'

    return styles

In [ ]:
age_bins = {
    "6–11": (6, 11),
    "12–18": (12, 18),
    "19–59": (19, 59),
    "60+": (60, 100),
}

genders = ["F", "M"]
speech_styles = ["read", "spontaneous"]

def get_age_bin(age):
    for label, (low, high) in age_bins.items():
        if low <= age <= high:
            return label
    return None

def get_accent(region, native):
    if native == "yes":
        return region
    else:
        return "Non-native"
    

def get_wer_by_demographic(speaker_info, speaker_performance):
    #structure: [accent][age_bin][style][gender]
    data = defaultdict(
        lambda: defaultdict(
            lambda: defaultdict(
                lambda: defaultdict(lambda: {'wer_sum': 0.0, 'words_sum': 0})
            )
        )
    )
    nr_dropped = 0
    #merge metadata and performance
    for _, row in speaker_info.iterrows():
        speaker_id = row['speaker_id'].strip().lower()
        sys_filename = row['sys_filename'].strip().lower()

        perf = speaker_performance.get(speaker_id)
        if perf is None:
            nr_dropped += 1
            print(f"No performance data for {sys_filename} speaker {speaker_id}")
            continue

        wer = perf['wer']
        num_words = perf['num_words']

        if not isinstance(row['age'], (int, float)):
            nr_dropped += 1
            print(f"Invalid age for {sys_filename} speaker {speaker_id}: {row['age']}")
            continue

        age_bin = get_age_bin(row['age'])
        if age_bin is None:
            nr_dropped += 1
            print(f"Invalid age bin for {sys_filename} speaker {speaker_id}: {row['age']}")
            continue

        accent = get_accent(row['region'], row['native'])

        gender = row['gender']
        if gender not in genders:
            nr_dropped += 1
            print(f"This is the gender police, you're under arrest: {sys_filename} speaker {speaker_id}: {gender}")
            continue

        if speaker_id.endswith('r'):
            speech_style = 'read'
        elif speaker_id.endswith('s'):
            speech_style = 'spontaneous'
        else:
            nr_dropped += 1
            print(f"Invalid speech style for {sys_filename} speaker {speaker_id}")
            continue

        entry = data[accent][age_bin][speech_style][gender]
        entry['wer_sum'] += wer * num_words
        entry['words_sum'] += num_words

    rows = []
    for accent in ["Dutch", "Flemish", "Non-native"]:
        for age_bin in age_bins:
            row = {}
            for style in speech_styles:
                for gender in genders:
                    cell = data[accent][age_bin][style][gender]
                    wer = cell['wer_sum'] / cell['words_sum'] if cell['words_sum'] > 0 else None
                    row[f"{style}_{gender}"] = wer
                #style average across genders
                total_words = sum(data[accent][age_bin][style][g]['words_sum'] for g in genders)
                total_wer = sum(data[accent][age_bin][style][g]['wer_sum'] for g in genders)
                row[f"{style}_Avg"] = total_wer / total_words if total_words > 0 else None
            row['Accent'] = accent
            row['Age Bin'] = age_bin
            rows.append(row)

    df = pd.DataFrame(rows)

    df = df.set_index(['Accent', 'Age Bin'])

    columns = []
    for style in speech_styles:
        columns += [f"{style}_F", f"{style}_M", f"{style}_Avg"]
    df = df[columns]

    wer_columns = [f"{style}_{g}" for style in speech_styles for g in genders] + [f"{style}_Avg" for style in speech_styles]
    df = df.dropna(subset=wer_columns, how='all')

    ordered_age_bins = list(age_bins.keys()) + ["Avg"]
    age_bin_order = pd.CategoricalDtype(ordered_age_bins, ordered=True)

    #compute average rows for each accent group
    avg_rows = []
    for accent in ["Dutch", "Flemish", "Non-native"]:

        sub_df = df.loc[accent]
        avg_row = sub_df.sum(numeric_only=True)
        counts = sub_df.notna().sum()
        avg_row = avg_row / counts
        avg_index = pd.MultiIndex.from_tuples([(accent, "Avg")], names=["Accent", "Age Bin"])
        avg_rows.append(pd.DataFrame([avg_row], index=avg_index))

    df = pd.concat([df] + avg_rows)
    df = df.reset_index()
    df["Age Bin"] = df["Age Bin"].astype(age_bin_order)
    df = df.sort_values(["Accent", "Age Bin"]).set_index(["Accent", "Age Bin"])
    df = df.round(1)
    df = df.dropna(subset=wer_columns, how='all')

    styled_df = df.style.apply(highlight_extremes, axis=None).format("{:.1f}")
    display(styled_df)

In [ ]:
accent_regions = ['n4a', 'n2c', 'n3b', 'fl4', 'fl3', 'fl2', 'fl1']
age_bins = {
    "6–11": (6, 11),
    "12–18": (12, 18),
    "61+": (60, 100),
}

def get_wer_by_accent(speaker_metadata, speaker_performance):
    data = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: {"wer_sum": 0.0, "words_sum": 0})))
    nr_dropped = 0

    #merge speaker data with performance
    for _, row in speaker_metadata.iterrows():
        speaker_id = row['speaker_id'].strip().lower()
        sys_filename = row['sys_filename'].strip().lower()

        if row['native'] != "yes":
            continue

        perf = speaker_performance.get(speaker_id)
        if perf is None:
            nr_dropped += 1
            print(f"No performance data for {sys_filename} speaker {speaker_id}")
            continue

        if not isinstance(row['age'], (int, float)):
            nr_dropped += 1
            print(f"Invalid age for {sys_filename} speaker {speaker_id}: {row['age']}")
            continue

        age_bin = get_age_bin(row['age'])
        if age_bin is None:
            print(f"Invalid age bin for {sys_filename} speaker {speaker_id}: {row['age']}")
            nr_dropped += 1
            continue

        if speaker_id.endswith('r'):
            style = 'read'
        elif speaker_id.endswith('s'):
            style = 'spontaneous'

        accent = row['accent_region'].lower()

        if style not in speech_styles or accent not in accent_regions:
            nr_dropped += 1
            continue

        wer = perf['wer']
        num_words = perf['num_words']

        entry = data[accent][age_bin][style]
        entry["wer_sum"] += wer * num_words
        entry["words_sum"] += num_words

    rows = []
    for style in speech_styles:
        for age_bin in age_bins:
            row = {"Style": style, "Age Bin": age_bin}
            wer_values = []
            for accent in accent_regions:
                cell = data[accent][age_bin][style]
                wer = cell["wer_sum"] / cell["words_sum"] if cell["words_sum"] > 0 else None
                row[accent] = wer
                if wer is not None:
                    wer_values.append(wer)
            #row average
            row["Avg"] = sum(wer_values) / len(wer_values) if wer_values else None
            rows.append(row)

    df = pd.DataFrame(rows)
    df.set_index(["Style", "Age Bin"], inplace=True)

    #column averages
    df.loc[("Avg", ""), :] = df.mean(numeric_only=True)
    df = df.round(2)
    
    styled_df = df.style.apply(highlight_extremes, axis=None).format("{:.1f}")
    display(styled_df)

In [ ]:
for subfolder in os.listdir(out_path):
    sub_metadata = metadata[(metadata.loc[:, 'sys_filename'].str.split("_").str[0] == subfolder)]
    sub_speaker_performance = {}
    
    for subset in range(1, 5):
        key = f"{subfolder}_{subset}_filtered.ctm.sys"
        perf_dict = speaker_performance.get(key)

        if perf_dict is None:
            print(f"Missing performance data for: {key}")
        else:
            sub_speaker_performance.update(perf_dict)

    print(f"Analyzing subset: {subfolder}")
    get_wer_by_accent(sub_metadata, sub_speaker_performance)

Analyzing subset: aws-transcribe


Analyzing subset: wav2vec2-dutch-large-ft-cgn


Analyzing subset: wav2vec2-large-xlsr-53-dutch


Analyzing subset: whisper-base


Analyzing subset: whisper-large-v3


Analyzing subset: whisper-large-v3-turbo


Analyzing subset: whisper-medium


Analyzing subset: whisper-small


Analyzing subset: whisper-tiny


In [ ]:
for subfolder in os.listdir(out_path):
    sub_metadata = metadata[(metadata.loc[:, 'sys_filename'].str.split("_").str[0] == subfolder)]
    sub_speaker_performance = {}
    
    for subset in range(1, 5):
        key = f"{subfolder}_{subset}_filtered.ctm.sys"
        perf_dict = speaker_performance.get(key)

        if perf_dict is None:
            print(f"Missing performance data for: {key}")
        else:
            sub_speaker_performance.update(perf_dict)

    print(f"Analyzing subset: {subfolder}")
    get_wer_by_demographic(sub_metadata, sub_speaker_performance)

Analyzing subset: aws-transcribe


Analyzing subset: wav2vec2-dutch-large-ft-cgn


Analyzing subset: wav2vec2-large-xlsr-53-dutch


Analyzing subset: whisper-base


Analyzing subset: whisper-large-v3


Analyzing subset: whisper-large-v3-turbo


Analyzing subset: whisper-medium


Analyzing subset: whisper-small


Analyzing subset: whisper-tiny
